In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(temperature=0.1)

cache_dir = LocalFileStore(".cache/")

memory = ConversationBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = TextLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        # MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

# chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm
chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)


def invoke_chain(question):
    result = chain.invoke(
        {
            "context": retriever,
            "question": question,
        }
    )
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [2]:
chain.invoke("Is Aaronson guilty?")

AIMessage(content='According to the document, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.')

In [3]:
chain.invoke("What message did he write in the table?")

AIMessage(content='He wrote "FREEDOM IS SLAVERY" and then "TWO AND TWO MAKE FIVE" on the table.')

In [4]:
chain.invoke("Who is Julia?")

AIMessage(content="Julia is a character mentioned in the text. She is someone who Winston has a close relationship with and has not betrayed, according to O'Brien.")

In [5]:
invoke_chain("Is Aaronson guilty?")

TypeError: expected string or buffer